# Historic Vital Signs Data 


After a little bit of manual cleaning, I got the historic data (2000-2010) for many indicators into a common format, stored in several different excel files with several sheets each. Next step is to compile everything into a dataframe that's easier to work with. 

In [1]:
import pandas as pd
import os # for identifying which files to read
import requests
import geopandas as gpd
from geopandas import GeoDataFrame

ModuleNotFoundError: No module named 'geopandas'

### Testing how to clean / compile the historic indicators from excel files - SKIP 

Starting with just testing things out.


Goal is to write some functions to do this stuff quickly, but first let me see how things go with a single excel file

In [ ]:
# get file names - only read the "clean" data
filenames = os.listdir("Vital-Signs-10-Data-Tables/Vital Signs 10 Data Tables")
cleanfiles = [filename for filename in filenames if "clean" in filename]
cleanpaths = ["Vital-Signs-10-Data-Tables/Vital Signs 10 Data Tables/" + filename for filename in cleanfiles]

In [ ]:
cleanfiles

In [ ]:
cleanpaths

In [ ]:
# test with one file
test = pd.ExcelFile(cleanpaths[0])
test.sheet_names

In [ ]:
sheet_to_df_map = {}
for sheet_name in test.sheet_names:
    sheet_to_df_map[sheet_name] = test.parse(sheet_name)

In [ ]:
pop_female = sheet_to_df_map["femaleXX"]
pop_male = sheet_to_df_map["maleXX"]

In [ ]:
pop_female.columns

In [ ]:
pop_female

In [ ]:
# melt example
pop_female_melted = pop_female.melt(id_vars = "Community Statistical Area (CSA)",
                         var_name = "year",
                         value_name = "femaleXX")\
.rename(columns={"Community Statistical Area (CSA)":"CSA"})
pop_female_melted

### Function to read / compile a single excel file into a dataframe  - SKIP 

Just load the previously compiled CSV now that I've already put it all together. 

In [ ]:
def compile_sheets(excel_path):
    # read every sheet of the excel, store as a dicitonary of dataframes
    xl = pd.ExcelFile(excel_path)
    sheet_map = {}
    for sheet_name in xl.sheet_names:
        sheet_map[sheet_name] = xl.parse(sheet_name)
        # standardize one particular column name
        sheet_map[sheet_name] = sheet_map[sheet_name].rename(columns = {"2006-2010": "2006 - 2010",
                                                                       "2006- 2010": "2006 - 2010"})
    # turn dictionary of dataframes into a single dataframe
    df = pd.concat(sheet_map).reset_index()\
    .rename(columns = {"level_0":"indicator", "Community Statistical Area (CSA)":"CSA"})\
    .drop(['level_1'],axis = 1)
    # set indicator category based on file name 
    df["indicator_category"] = str(excel_path[65:-5])
    return df

In [ ]:
# compile all indicators across all clean historic excel files
results = [compile_sheets(path) for path in cleanpaths]
historic_indicators = pd.concat(results)

In [ ]:
# rearrange the columns
historic_indicators = historic_indicators[['CSA',
                                         'indicator',
                                         'indicator_category',
                                         2000,
                                         2001,
                                         2002,
                                         2003,
                                         2004,
                                         2005,
                                         2006,
                                         2007,
                                         2008,
                                         2009,
                                         2010,
                                         '2006 - 2010'
                                        ]]

In [ ]:
historic_indicators

In [ ]:
# write to csv so we can just load this file going forward 
historic_indicators.to_csv("precompiled_historic_indicators.csv",index=False)

# Vital Signs from 2010 onward (attempt 1 - *failed*)

Making use of BNIA's API

Update 1/21 - don't use this section. Keeping it for posterity, but something about my function was causing the kernel to die whenever I made multiple API pulls

In [ ]:
# making use of previously created functions
def getGDFfromURL(url, layer=0):
    #GDF stands for GeoDataFrame; this is the innermost function called by getGDF
    tail = "/"+str(layer)+"/query?where=1%3D1&outFields=*&outSR=4326&f=json" #worked this out
    url+=tail
    print(url)
    gdf = gpd.read_file(url) #GeoPandas has a built in function to read APIs given right URL
    return gdf

def getGDF(shortname, level=0):
    #This is outermost function called by user; it calls getGDFfromURL
    url = api_df.loc[shortname, "API"]
    return getGDFfromURL(url, level)

In [ ]:
# pull in info about the available indicators 
api_df = pd.read_csv("VS-Indicator-APIs.csv")
api_df.set_index("ShortName", inplace=True) #Makes the dataframe index the shortname

In [ ]:
api_df

In [ ]:
# grab a few indicators to loop through while testing
indicator_sample = ["paaXX","pwhiteXX", "phispXX"]

#### Reformatting 1 indicator

In [ ]:
## reformatting 1 indicator 
paaXX_df = getGDF("paaXX")

In [ ]:
# add column for indicator 
#paaXX_df["indicator"] = "paaXX"
# remove object ID 
# change colnames to years (by index, but needs to be sensitive to specific year)
# melt 

In [ ]:
paaXX_df = paaXX_df.drop(["OBJECTID"], axis = 1)

In [ ]:
['20' + i[-2:] for i in paaXX_df.columns[1:-3]]

In [ ]:
# get list of years to use as column names
yrs = [int('20' + i[-2:]) for i in paaXX_df.columns[1:-3]]
yrs

In [ ]:
# replace column names
for i in range(0,len(yrs)):
    paaXX_df.columns.values[1+i] = yrs[i]

In [ ]:
# what are the index variables? 
z = [i for i in paaXX_df.columns[-3:]]
z.insert(0,paaXX_df.columns[0])

In [ ]:
z

In [ ]:
# melt 
paaXX_df.melt(id_vars = z, 
             var_name = "year", 
             value_name = "paaXX")

#### Try to loop and append a few dataframes

In [ ]:
### SEEMS TO WORK, BUT KERNAL KEEPS DYING! 
# function draft space 
def compile_post10_indicators_draft(indicator_list):
    ### first indicator (retain CSA and geometry fields)
    # get geopanadas dataframe from API
    first_result = getGDF(indicator_list[0])
    # drop object ID column 
    first_result = first_result.drop(["OBJECTID"], axis = 1)
    # get list of years (columns) present in the data
    first_yrs = [int('20' + i[-2:]) for i in first_result.columns[1:-3]]
    # replace existing column names with actual year values 
    for i in range(0,len(first_yrs)):
        first_result.columns.values[1+i] = first_yrs[i]
    # melt (pivot longer) to a final dataframe 
    first_index_vars = [i for i in first_result.columns[-3:]]
    first_index_vars.insert(0, first_result.columns[0])
    df = first_result.melt(id_vars = first_index_vars, 
                                var_name = "year", 
                                value_name = indicator_list[0])
    ### loop through this process for other indicators (and remove the geometry), and full join 
    for indicator in indicator_list[1:]:
        result = getGDF(indicator).drop(["OBJECTID"], axis = 1)
        yrs = [int('20' + i[-2:]) for i in result.columns[1:-3]]
        for i in range(0,len(yrs)):
            result.columns.values[1+i] = yrs[i]
        index_vars = [i for i in result.columns[-3:]]
        index_vars.insert(0, result.columns[0])
        df_indicator = result.melt(id_vars = index_vars, 
                                   var_name = "year", 
                                   value_name = indicator)
        df_indicator = df_indicator.iloc[:,[0,4,5]]  # drops the geometry columns 
        df = pd.merge(df, df_indicator, how = "outer", on = "CSA2010")
        
    
    
    return df
        

In [ ]:
## trying again. but this time, built for a single indicator 
def grab_reformat_indicator(indicator):
    result = getGDF(indicator)
    result = result.drop(["OBJECTID"], axis = 1)
    yrs = [int('20' + i[-2:]) for i in result.columns[1:-3]]
    for i in range(0,len(yrs)):
        result.columns.values[1+i] = yrs[i]
    index_vars = [i for i in result.columns[-3:]]
    index_vars.insert(0, result.columns[0])
    df = result.melt(id_vars = index_vars,
                     var_name = "year",
                     value_name = indicator)
    return df


In [ ]:
indicator_sample[1:]

In [ ]:
compile_post10_indicators(indicator_sample[:2])

In [ ]:
d1 = compile_post10_indicators(["paaXX"])
d2 = compile_post10_indicators(["pwhiteXX"])

In [ ]:
d2

In [ ]:
d2_2 = d2.iloc[:,[0,4,5]]

In [ ]:
d2_2

In [ ]:
pd.merge(d1,d2_2, how = "outer")

Things were going okay, but my kernel kept dying when I tried to run my function on multiple indicators.
Rather than keep trying with this method, I went back to Colin's BNIA code and I'm going to make use of his 'collect' function

# Vital Signs from 2010 onwards, attempt 2 

Making use of Colin's existing functions (I know they work, and I'm trying to avoid killing the kernel) 

In [ ]:
conda install geopandas

In [ ]:
# making use of previously created functions
def getGDFfromURL(url, layer=0):
    #GDF stands for GeoDataFrame; this is the innermost function called by getGDF
    tail = "/"+str(layer)+"/query?where=1%3D1&outFields=*&outSR=4326&f=json" #worked this out
    url+=tail
    print(url)
    gdf = gpd.read_file(url) #GeoPandas has a built in function to read APIs given right URL
    return gdf

def getGDF(shortname, level=0):
    #This is outermost function called by user; it calls getGDFfromURL
    url = api_df.loc[shortname, "API"]
    return getGDFfromURL(url, level)

def getCollect(check_list):
    #This function collects all the target GDFs and puts into collection
    collect=[]
    for shortname in check_list:
        gdf=getGDF(shortname)
        collect.append(gdf)    
    return collect

Rather than pull every single indicator, I've pared down the list a bit to remove indicators that aren't usable or wouldn't add anything particularly interesting to the analysis. Some indicators weren't very useful because the metric was collected by zip code or because the data was otherwise very sparse. 

In [ ]:
# read list of indicators 
api_df = pd.read_csv("VS-Indicator-APIs_EF.csv") # new version - I've labeled which API calls to make under 'pull'
#api_df.set_index("ShortName", inplace=True) 
api_df

In [ ]:
# how many API calls are we making? 
sum(api_df.pull)

In [ ]:
### API calls 
# get list of shortnames indicators to pull 
indicator_list = list(api_df[api_df.pull == 1].index)
# collect (first 50)
collect1 = getCollect(indicator_list[:50])

In [ ]:
# split up in case of errors along the way
collect2 = getCollect(indicator_list[50:100])

In [ ]:
collect3 = getCollect(indicator_list[100:])

In [ ]:
# turn them into dataframes and append 
df1 = pd.concat(collect1)
df2 = pd.concat(collect2)
df3 = pd.concat(collect3)

In [ ]:
# final dataframe of vital signs from 2010 onward (from 137 selected indicators)
vs = df1.append(df2).append(df3)
vs

In [ ]:
# check that the append worked properly, and we have the right number of rows 
df1.shape[0] + df2.shape[0] + df3.shape[0]

In [ ]:
# export to csv (temporary file, to avoid repulling the data)
df1.to_csv("modern_vital_signs_raw_1.csv", index = False)
df2.to_csv("modern_vital_signs_raw_2.csv", index = False)
df3.to_csv("modern_vital_signs_raw_3.csv", index = False)

#### Reformat (modern) Data 

- Pivot: create a new column for indicator name, change column names from indicator + year to just year 

In [ ]:
def reformat_indicator(collect_object):
    collect_object = collect_object.iloc[: , 1:] # drop the first column, which is object ID 
    yrs = ['20' + i[-2:] for i in collect_object.columns[1:-3]]
    indicator = collect_object.columns[2][:-2]
    for i in range(0,len(yrs)):
        collect_object.columns.values[1+i] = yrs[i]
    collect_object["indicator"] = indicator
    collect_object = collect_object.reset_index()
    return collect_object

In [ ]:
# difference from last cell: the 'if' statement for objectID 
def reformat_indicator_2(collect_object):
    if "OBJECTID" in collect_object.columns:
        collect_object = collect_object.drop(["OBJECTID"], axis = 1)
    yrs = ['20' + i[-2:] for i in collect_object.columns[1:-3]]
    indicator = collect_object.columns[2][:-2]
    for i in range(0,len(yrs)):
        collect_object.columns.values[1+i] = yrs[i]
    collect_object["indicator"] = indicator
    collect_object = collect_object.reset_index()
    return collect_object

In [ ]:
# make one giant collection: 
full_collection = collect1.copy()
[full_collection.append(c) for c in collect2]
[full_collection.append(c) for c in collect3]
len(full_collection) # now the correct length

In [ ]:
# loop for all of collect 1 (which is now everything) 
# COMMENTED OUT, KERNEL WAS DYING 
# reformatted_dfs = []
# for c in full_collection: 
#     data = reformat_indicator(c).reset_index(drop = True)
#     reformatted_dfs.append(data)

In [ ]:
# make dataframes out of collections 
### KERNEL KEEPS DYING! I'll go ahead and split this into many different files
reformatted_dfs_1 = []
for c in collect1[:25]: 
    reformatted_dfs_1.append(reformat_indicator(c))

In [ ]:
dfs_1 = pd.concat(reformatted_dfs_1)

In [ ]:
dfs_1.to_csv("modern_vital_signs_1.csv", index = False)

In [ ]:
reformatted_dfs_2 = []
for c in collect2: 
    data = reformat_indicator(c).reset_index()
    reformatted_dfs_2.append(data)

In [ ]:
reformatted_dfs_3 = []
for c in collect3: 
    data = reformat_indicator(c).reset_index()
    reformatted_dfs_3.append(data)

In [ ]:
reformatted_dfs

### Kernel keeps dying! Work around approach.... 


Reading the previously saved data, even though it's in a terrible format. From there: 
- melt so that we have columns for neighborhood, geometry, and the metric itself. each indicator/year column will become a new row 
- add new column for year, based on indicator 
- add new column for indicator (agnostic of year) 
- drop indicator/year column 
- re-save data 

In [ ]:
# read in data files for modern indicators 
mvs1 = pd.read_csv("modern_vital_signs_raw_1.csv")
mvs2 = pd.read_csv("modern_vital_signs_raw_2.csv")
mvs3 = pd.read_csv("modern_vital_signs_raw_3.csv")

In [ ]:
## reformat / melt 
# msv1 
objectid_cols = [col for col in mvs1.columns if "OBJECTID" in col]

mvs1 = mvs1.drop(objectid_cols, axis = 1)
# drop geometry as well; it's causing some problems
mvs1 = mvs1.drop(['Shape__Area', 'Shape__Length', "geometry"], axis = 1)
index_cols = ["CSA2010"]
mvs_df_1 = mvs1.melt(id_vars = index_cols,
                     var_name = "year-indicator",
                     value_name = "value")
# msv2 
objectid_cols = [col for col in mvs2.columns if "OBJECTID" in col]
mvs2 = mvs2.drop(objectid_cols, axis = 1)
mvs2 = mvs2.drop(['Shape__Area', 'Shape__Length', "geometry"], axis = 1)
mvs_df_2 = mvs2.melt(id_vars = index_cols,
                     var_name = "year-indicator",
                     value_name = "value")
# msv3
objectid_cols = [col for col in mvs3.columns if "OBJECTID" in col]
mvs3 = mvs3.drop(objectid_cols, axis = 1)
mvs3 = mvs3.drop(['Shape__Area', 'Shape__Length', "geometry"], axis = 1)
mvs_df_3 = mvs3.melt(id_vars = index_cols,
                     var_name = "year-indicator",
                     value_name = "value")

In [ ]:
## add column for year, based on indicator/year field 
mvs_df_1["year"] = ['20' + i[-2:] for i in mvs_df_1["year-indicator"]]
mvs_df_2["year"] = ['20' + i[-2:] for i in mvs_df_2["year-indicator"]]
mvs_df_3["year"] = ['20' + i[-2:] for i in mvs_df_3["year-indicator"]]
## add column for indicator, based on indicator/year field 
mvs_df_1["indicator"] = [i[:-2] for i in mvs_df_1["year-indicator"]]
mvs_df_2["indicator"] = [i[:-2] for i in mvs_df_2["year-indicator"]]
mvs_df_3["indicator"] = [i[:-2] for i in mvs_df_3["year-indicator"]]
# drop year-indicator field 
mvs_df_1 = mvs_df_1.drop(["year-indicator"], axis = 1)
mvs_df_2 = mvs_df_2.drop(["year-indicator"], axis = 1)
mvs_df_3 = mvs_df_3.drop(["year-indicator"], axis = 1)

In [ ]:
# somehow ended up with a lot of duplicates because of NA values. Drop those 
mvs_df_1.dropna(subset = ["value"], inplace = True)
mvs_df_2.dropna(subset = ["value"], inplace = True)
mvs_df_3.dropna(subset = ["value"], inplace = True)
# it also seems like there are some indicators where the API failed to pull data, resulting in NAs in CSA2010 
mvs_df_1.dropna(subset = ["CSA2010"], inplace = True)
mvs_df_2.dropna(subset = ["CSA2010"], inplace = True)
mvs_df_3.dropna(subset = ["CSA2010"], inplace = True)

In [ ]:
mvs_df_1

In [ ]:
## pivot 
index_cols_pivotlonger = ["CSA2010", "indicator"]
mvs_pivot_1 = mvs_df_1.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()
mvs_pivot_2 = mvs_df_2.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()
mvs_pivot_3 = mvs_df_3.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()

In [ ]:
### small data fix - remove some placeholder rows from mvs3 
mvs_df_3 = mvs_df_3.query("indicator != 'CSA2010'")

In [ ]:
## export! 
mvs_pivot_1.to_csv("modern_vital_signs_pivot_1.csv", index = False)
mvs_pivot_2.to_csv("modern_vital_signs_pivot_2.csv", index = False)
mvs_pivot_3.to_csv("modern_vital_signs_pivot_3.csv", index = False)

#### random troubleshooting below

In [ ]:
index_cols_pivotlonger = mvs_df_1.columns[:4]

In [ ]:
index_cols_pivotlonger = [i for i in index_cols_pivotlonger]

In [ ]:
index_cols_pivotlonger.append("indicator")

In [ ]:
index_cols_pivotlonger

In [ ]:
## pivot 
# mvs_pivot_1 = mvs_df_1.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()
# mvs_pivot_2 = mvs_df_2.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()
# mvs_pivot_3 = mvs_df_3.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()

In [ ]:
## export! 
# mvs_pivot_1.to_csv("modern_vital_signs_pivot_1.csv", index = False)
# mvs_pivot_2.to_csv("modern_vital_signs_pivot_2.csv", index = False)
# mvs_pivot_3.to_csv("modern_vital_signs_pivot_3.csv", index = False)

In [ ]:
# idk why there are duplicates, let's look for them 
groupby_cols = index_cols_pivotlonger.copy()
groupby_cols.append("year")
grouped = mvs_df_1.groupby(groupby_cols).size().reset_index()

In [ ]:
grouped

In [ ]:
# dat.loc[dat.lifeExp < 25]
# dat.query('lifeExp < 25')
viol = mvs_df_1.query("indicator == 'viol'")
viol.query("year == '2016'").dropna(subset = ["value"], inplace =True)

In [ ]:
mvs_df_1.query("indicator == 'demper'")

In [ ]:
# there are a bunch of NA's in CSA2010 for 'demper' - why? 
demper_cols =[col for col in mvs1.columns if 'demper' in col]

In [ ]:
mvs1[demper_cols] # I'm assuming the API failed for this indicator

In [ ]:
mvs1[mvs1.CSA2010.isnull()]

In [ ]:
mvs2[mvs2.CSA2010.isnull()]

In [ ]:
mvs3[mvs3.CSA2010.isnull()]

# Combining Historic & Modern Vital Sign Data

In [ ]:
### historic data 
# read from CSV 
historic_indicators = pd.read_csv("precompiled_historic_indicators.csv")
# pivot longer for join 
hvs = historic_indicators.melt(id_vars = ["CSA", "indicator", "indicator_category"], 
                        var_name = "year")
# drop indicator category (will add it later so that it's uniform)
hvs = hvs.drop(["indicator_category"], axis = 1)

In [ ]:
### modern data 
# reshape (longer pivot makes more sense for union )
mvs_df_1 = mvs_df_1.rename(columns = {"CSA2010":"CSA"})
mvs_df_2 = mvs_df_2.rename(columns = {"CSA2010":"CSA"})
mvs_df_3 = mvs_df_3.rename(columns = {"CSA2010":"CSA"})


In [ ]:
### concatenate into one big dataframe 
vs = pd.concat([hvs, mvs_df_1,mvs_df_2,mvs_df_3])

In [ ]:
# clean some names (remove asterisk from CSA names)
vs["CSA"] = vs.CSA.str.replace("*","", regex = False)

In [ ]:
# export for posterity 
vs.to_csv("full_vital_signs.csv", index = False)

In [ ]:
# what years are available for each indicator? 
vs[["indicator","year"]].groupby(["indicator"])["year"].apply(set).reset_index()
# just from a glance, it looks like there are some duplicates here

In [ ]:
# create an info dataframe of indicator, years available, description, and category 
info = vs[["indicator","year"]].groupby(["indicator"])["year"].apply(set).reset_index()
# grab the info from the api DF
indicator_desc = api_df.rename(columns = {"Indicator":"indicator_description","ShortName":"indicator","Section":"category"})[["indicator_description","indicator","category"]]
info = info.merge(indicator_desc)
info

In [ ]:
# export 
info.to_csv("indicator_info.csv", index = False)

In [ ]:
# check that all the neighborhood names are uniform 
len(set(vs.CSA)) # there should be about 55, so there's cleanup to do 

In [ ]:
len(set(mvs_df_1.CSA))

In [ ]:
## clean some names 
# remove asterisk from CSA names
# correct some spellings 
# unify some names that may be abbreviated
vs["CSA"] = vs.CSA.str.replace("*","", regex = False)
vs["CSA"] = vs.CSA.str.replace("Edmonson","Edmondson")
vs["CSA"] = vs.CSA.str.replace("Falstaff","Fallstaff") # really not sure which is right, but BNIA uses Fallstaff in modern communications
vs["CSA"] = vs.CSA.str.replace("Mt. Washington","Mount Washington")
vs["CSA"] = vs.CSA.str.replace("Mt. Winans","Mount Winans")

In [ ]:
set(vs.CSA)
# things that need to get cleaned up: 
#* 2010 data using new 2010 CSA boundaries. CSA boundaries were modified slightly due to modifications in Census geographies from 2000 to 2010.
# For more information, visit http://www.bniajfi.org.
# NA = Data not available due to major modifications in Census geographies from 2000 to 2010. 
# nan 
# anything with a * 
# Edmonson Village vs Edmondson Village
# Glen-Fallstaff vs. Glen-Falstaff
# Jonestown/Oldtown vs. Oldtown / Middle East
# Washington Village vs. Washington Village/Pigtown
# Westport/Mount Winans/Lakeland vs Westport/Mt. Winans/Lakeland
# Perkins/Middle East vs Oldtown/Middle East
# 'Medfield/Hampden/Woodberry', vs 'Medfield/Hampden/Woodberry/Remington'
#  'Mount Washington/Coldspring','Mt. Washington/Coldspring',

In [ ]:
## TO DO: check the api call for "Hhsize" 
# modern data 
# does not seem to deliver a year, which causes some weirdness in the data frame 